In [24]:
import time
from terraaws.utilities import S3
from terraaws.network import VPC
from terraaws.compute import EC2
from terraaws.database import RDS

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#project s3 bucket
s3 = S3(profile_name = 'mjn')
prj_bucket = s3.create_private_bucket('tripoli-env')

In [4]:
#project vpc, security group, route table, 2 subnets for database, public subnet
vpc = VPC(profile_name = 'mjn')
prj_vpc, prj_sg, prj_rt = vpc.create_base_vpc('213.213.48.0/24', 'tripoli')
subnets = vpc.create_subnet('213.213.48.32/27', 'us-west-2b', 'tripoli-dbsubnet-01')
subnets = vpc.create_subnet('213.213.48.64/27', 'us-west-2c', 'tripoli-dbsubnet-02')
subnets = vpc.create_subnet('213.213.48.96/27', 'us-west-2a', 'tripoli-publicsubnet', public = True)
intgateway = vpc.create_internet_gateway('tripoli-ig')
int_route = vpc.create_internet_route('0.0.0.0/0')

In [14]:
#amazon linux ec2
ec2 = EC2(profile_name = 'mjn')
prj_ec2 = ec2.launch_amazon_linux2('tripoli-ec2', publicsubnet['Subnets'][0]['SubnetId'], 
                                   [prj_sg['SecurityGroups'][0]['GroupId']],
                                  'tripoli_key')

In [28]:
#database subnet group, amazon mysql rds
rds = RDS(profile_name = 'mjn')
dbsubnetgrp = rds.create_db_subnet_group('tripolidb', 
                                         [publicsubnet['Subnets'][1]['SubnetId'], publicsubnet['Subnets'][2]['SubnetId']])


In [35]:
prj_db = rds.launch_mysql_rds('tripoliwordpress', 'tripoliadmin', 'Tripoliwp2021!', 
                 dbsubnetgrp['DBSubnetGroups'][0]['DBSubnetGroupName'], 
                 prj_sg['SecurityGroups'][0]['GroupId'])

AttributeError: 'RDS' object has no attribute 'client'

In [ ]:
#write configs
put_status = s3.put_object(prj_bucket, 'tripoli-env', 'tripoli-s3-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_vpc, 'tripoli-env', 'tripoli-vpc-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_sg, 'tripoli-env', 'tripoli-sg-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(publicsubnet, 'tripoli-env', 'tripoli-subnet-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(intgateway, 'tripoli-env', 'tripoli-intgateway-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(int_route, 'tripoli-env', 'tripoli-route-table-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_ec2, 'tripoli-env', 'tripoli-ec2-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(dbsubnetgrp, 'tripoli-env', 'tripoli-dbsubnet-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_db, 'tripoli-env', 'tripoli-db-config-' + str(int(time.time())) + '.json')